- 需求：
    - 导入文件，查看原始数据
    - 将人口数据和各州简称数据进行合并
    - 将合并的数据中重复的abbreviation列进行删除
    - 查看存在缺失数据的列
    - 找到有哪些state/region使得state的值为NaN，进行去重操作
    - 为找到的这些state/region的state项补上正确的值，从而去除掉state这一列的所有NaN
    - 合并各州面积数据areas
    - 我们会发现area(sq.mi)这一列有缺失数据，找出是哪些行
    - 去除含有缺失数据的行
    - 找出2010年的全民人口数据
    - 计算各州的人口密度
    - 排序，并找出人口密度最高的州

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame

In [2]:
#导入文件，查看原始数据
abb = pd.read_csv('./data/state-abbrevs.csv') #state(州的全称)abbreviation（州的简称）

In [5]:
abb.head(1) #州全称和简称

state abbreviation
0  Alabama           AL

In [6]:
area = pd.read_csv('./data/state-areas.csv') #state州的全称，area (sq. mi)州的面积
area.head(1) #州的全称和面积

state  area (sq. mi)
0  Alabama          52423

In [7]:
pop = pd.read_csv('./data/state-population.csv')#state/region简称，ages年龄，year时间，population人口数量
pop.head(1)# 州的简称，年龄层次，年份，人口数量

state/region     ages  year  population
0           AL  under18  2012   1117489.0

In [8]:
#将人口数据和各州简称数据进行合并
abb_pop = pd.merge(abb,pop,left_on='abbreviation',right_on='state/region',how='outer')
abb_pop.head()

state abbreviation state/region     ages  year  population
0  Alabama           AL           AL  under18  2012   1117489.0
1  Alabama           AL           AL    total  2012   4817528.0
2  Alabama           AL           AL  under18  2010   1130966.0
3  Alabama           AL           AL    total  2010   4785570.0
4  Alabama           AL           AL  under18  2011   1125763.0

In [9]:
abb_pop.drop(labels='abbreviation',axis=1,inplace=True)

In [10]:
#查看存在缺失数据的列
#方式1：
abb_pop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2544 entries, 0 to 2543
Data columns (total 5 columns):
state           2448 non-null object
state/region    2544 non-null object
ages            2544 non-null object
year            2544 non-null int64
population      2524 non-null float64
dtypes: float64(1), int64(1), object(3)
memory usage: 119.2+ KB


In [11]:
#方式2
abb_pop.isnull().any(axis=0)

state            True
state/region    False
ages            False
year            False
population       True
dtype: bool

In [29]:
#找到有哪些state/region使得state的值为NaN，进行去重操作
#1.找到state中的空值
abb_pop['state'].isnull()
#2.将空值对应的行数据提取
abb_pop.loc[abb_pop['state'].isnull()]
#3.将state空对应的简称找出
abb_pop.loc[abb_pop['state'].isnull()]['state/region']
#4.对符合要求的简称进行去重
abb_pop.loc[abb_pop['state'].isnull()]['state/region'].unique()

array([], dtype=object)

In [19]:
#nunique可以统计重复元素的个数
abb_pop.loc[abb_pop['state'].isnull()]['state/region'].nunique()

2

In [23]:
#为找到的这些state/region的state项补上正确的值，从而去除掉state这一列的所有NaN
#1.state这一列中的空值对应的简称为PR和USA，state这一列的空值可以分为两种类型，一种类型
#需要补充PR的全称，一种类型需要补充为USA的全称
#2.找到PR简称对应全称的空值
    #可以将PR对应的行数据取出，然后将state列取出即可
abb_pop['state/region'] == 'PR'
abb_pop.loc[abb_pop['state/region'] == 'PR']
#3.将符合要求的空进行正确填充
indexs = abb_pop.loc[abb_pop['state/region'] == 'PR'].index
abb_pop.loc[indexs,'state'] = 'PPPRRR'#批量赋值

In [28]:
abb_pop['state/region'] == 'USA'
abb_pop.loc[abb_pop['state/region'] == 'USA']
indexs = abb_pop.loc[abb_pop['state/region'] == 'USA'].index
abb_pop.loc[indexs,'state'] = 'United States'

In [ ]:
#合并各州面积数据areas
我们会发现area(sq.mi)这一列有缺失数据，找出是哪些行
去除含有缺失数据的行
找出2010年的全民人口数据
计算各州的人口密度
排序，并找出人口密度最高的州

In [30]:
#合并各州面积数据areas
area.head(1)

state  area (sq. mi)
0  Alabama          52423

In [31]:
abb_pop.head(1)

state state/region     ages  year  population
0  Alabama           AL  under18  2012   1117489.0

In [32]:
abb_pop_area = pd.merge(abb_pop,area,how='outer')

In [33]:
abb_pop_area.head()

state state/region     ages    year  population  area (sq. mi)
0  Alabama           AL  under18  2012.0   1117489.0        52423.0
1  Alabama           AL    total  2012.0   4817528.0        52423.0
2  Alabama           AL  under18  2010.0   1130966.0        52423.0
3  Alabama           AL    total  2010.0   4785570.0        52423.0
4  Alabama           AL  under18  2011.0   1125763.0        52423.0

In [36]:
#我们会发现area(sq.mi)这一列有缺失数据，找出是哪些行
abb_pop_area['area (sq. mi)'].isnull()
abb_pop_area.loc[abb_pop_area['area (sq. mi)'].isnull()]
drop_index = abb_pop_area.loc[abb_pop_area['area (sq. mi)'].isnull()].index
drop_index

Int64Index([2448, 2449, 2450, 2451, 2452, 2453, 2454, 2455, 2456, 2457, 2458,
            2459, 2460, 2461, 2462, 2463, 2464, 2465, 2466, 2467, 2468, 2469,
            2470, 2471, 2472, 2473, 2474, 2475, 2476, 2477, 2478, 2479, 2480,
            2481, 2482, 2483, 2484, 2485, 2486, 2487, 2488, 2489, 2490, 2491,
            2492, 2493, 2494, 2495, 2496, 2497, 2498, 2499, 2500, 2501, 2502,
            2503, 2504, 2505, 2506, 2507, 2508, 2509, 2510, 2511, 2512, 2513,
            2514, 2515, 2516, 2517, 2518, 2519, 2520, 2521, 2522, 2523, 2524,
            2525, 2526, 2527, 2528, 2529, 2530, 2531, 2532, 2533, 2534, 2535,
            2536, 2537, 2538, 2539, 2540, 2541, 2542, 2543],
           dtype='int64')

In [37]:
abb_pop_area.drop(labels=drop_index,axis=0,inplace=True)

In [42]:
#删除最后一行数据
abb_pop_area.drop(labels=2544,axis=0,inplace=True)

In [43]:
abb_pop_area.isnull().any(axis=0)

state            False
state/region     False
ages             False
year             False
population       False
area (sq. mi)    False
dtype: bool

In [45]:
abb_pop_area.head()

state state/region     ages    year  population  area (sq. mi)
0  Alabama           AL  under18  2012.0   1117489.0        52423.0
1  Alabama           AL    total  2012.0   4817528.0        52423.0
2  Alabama           AL  under18  2010.0   1130966.0        52423.0
3  Alabama           AL    total  2010.0   4785570.0        52423.0
4  Alabama           AL  under18  2011.0   1125763.0        52423.0

In [46]:
#找出2010年的全民人口数据
abb_pop_area.query('year == 2010 & ages == "total"')

state state/region   ages    year  population  \
3                  Alabama           AL  total  2010.0   4785570.0   
91                  Alaska           AK  total  2010.0    713868.0   
101                Arizona           AZ  total  2010.0   6408790.0   
189               Arkansas           AR  total  2010.0   2922280.0   
197             California           CA  total  2010.0  37333601.0   
283               Colorado           CO  total  2010.0   5048196.0   
293            Connecticut           CT  total  2010.0   3579210.0   
379               Delaware           DE  total  2010.0    899711.0   
389   District of Columbia           DC  total  2010.0    605125.0   
475                Florida           FL  total  2010.0  18846054.0   
485                Georgia           GA  total  2010.0   9713248.0   
570                 Hawaii           HI  total  2010.0   1363731.0   
581                  Idaho           ID  total  2010.0   1570718.0   
666               Illinois           IL  total  2010.0  12839695.0   
677                Indiana           IN  total  2010.0   6489965.0   
762                   Iowa           IA  total  2010.0   3050314.0   
773                 Kansas           KS  total  2010.0   2858910.0   
858               Kentucky           KY  total  2010.0   4347698.0   
869              Louisiana           LA  total  2010.0   4545392.0   
954                  Maine           ME  total  2010.0   1327366.0   
965                Montana           MT  total  2010.0    990527.0   
1050              Nebraska           NE  total  2010.0   1829838.0   
1061                Nevada           NV  total  2010.0   2703230.0   
1146         New Hampshire           NH  total  2010.0   1316614.0   
1157            New Jersey           NJ  total  2010.0   8802707.0   
1242            New Mexico           NM  total  2010.0   2064982.0   
1253              New York           NY  total  2010.0  19398228.0   
1338        North Carolina           NC  total  2010.0   9559533.0   
1349          North Dakota           ND  total  2010.0    674344.0   
1434                  Ohio           OH  total  2010.0  11545435.0   
1445              Oklahoma           OK  total  2010.0   3759263.0   
1530                Oregon           OR  total  2010.0   3837208.0   
1541              Maryland           MD  total  2010.0   5787193.0   
1626         Massachusetts           MA  total  2010.0   6563263.0   
1637              Michigan           MI  total  2010.0   9876149.0   
1722             Minnesota           MN  total  2010.0   5310337.0   
1733           Mississippi           MS  total  2010.0   2970047.0   
1818              Missouri           MO  total  2010.0   5996063.0   
1829          Pennsylvania           PA  total  2010.0  12710472.0   
1914          Rhode Island           RI  total  2010.0   1052669.0   
1925        South Carolina           SC  total  2010.0   4636361.0   
2010          South Dakota           SD  total  2010.0    816211.0   
2021             Tennessee           TN  total  2010.0   6356683.0   
2106                 Texas           TX  total  2010.0  25245178.0   
2117                  Utah           UT  total  2010.0   2774424.0   
2202               Vermont           VT  total  2010.0    625793.0   
2213              Virginia           VA  total  2010.0   8024417.0   
2298            Washington           WA  total  2010.0   6742256.0   
2309         West Virginia           WV  total  2010.0   1854146.0   
2394             Wisconsin           WI  total  2010.0   5689060.0   
2405               Wyoming           WY  total  2010.0    564222.0   

      area (sq. mi)  
3           52423.0  
91         656425.0  
101        114006.0  
189         53182.0  
197        163707.0  
283        104100.0  
293          5544.0  
379          1954.0  
389            68.0  
475         65758.0  
485         59441.0  
570         10932.0  
581         83574.0  
666         57918.0  
677         36420.0  
762         56276.0  
773   

In [48]:
#计算各州的人口密度
abb_pop_area['midu'] = abb_pop_area['population'] / abb_pop_area['area (sq. mi)']

In [49]:
abb_pop_area.head()

state state/region     ages    year  population  area (sq. mi)       midu
0  Alabama           AL  under18  2012.0   1117489.0        52423.0  21.316769
1  Alabama           AL    total  2012.0   4817528.0        52423.0  91.897221
2  Alabama           AL  under18  2010.0   1130966.0        52423.0  21.573851
3  Alabama           AL    total  2010.0   4785570.0        52423.0  91.287603
4  Alabama           AL  under18  2011.0   1125763.0        52423.0  21.474601

In [51]:
#对密度实现排序
abb_pop_area.sort_values(by='midu',axis=0,ascending=False)

state state/region     ages    year  population  \
391  District of Columbia           DC    total  2013.0    646449.0   
385  District of Columbia           DC    total  2012.0    633427.0   
387  District of Columbia           DC    total  2011.0    619624.0   
431  District of Columbia           DC    total  1990.0    605321.0   
389  District of Columbia           DC    total  2010.0    605125.0   
426  District of Columbia           DC    total  1991.0    600870.0   
429  District of Columbia           DC    total  1992.0    597567.0   
422  District of Columbia           DC    total  1993.0    595302.0   
392  District of Columbia           DC    total  2009.0    592228.0   
425  District of Columbia           DC    total  1994.0    589240.0   
420  District of Columbia           DC    total  1995.0    580519.0   
396  District of Columbia           DC    total  2008.0    580236.0   
406  District of Columbia           DC    total  2001.0    574504.0   
394  District of Columbia           DC    total  2007.0    574404.0   
408  District of Columbia           DC    total  2002.0    573158.0   
419  District of Columbia           DC    total  1996.0    572379.0   
412  District of Columbia           DC    total  2000.0    572046.0   
400  District of Columbia           DC    total  2006.0    570681.0   
410  District of Columbia           DC    total  1999.0    570220.0   
402  District of Columbia           DC    total  2003.0    568502.0   
404  District of Columbia           DC    total  2004.0    567754.0   
417  District of Columbia           DC    total  1997.0    567739.0   
398  District of Columbia           DC    total  2005.0    567136.0   
415  District of Columbia           DC    total  1998.0    565232.0   
421  District of Columbia           DC  under18  1995.0    123620.0   
424  District of Columbia           DC  under18  1994.0    122170.0   
418  District of Columbia           DC  under18  1996.0    121210.0   
423  District of Columbia           DC  under18  1993.0    120471.0   
416  District of Columbia           DC  under18  1997.0    119531.0   
428  District of Columbia           DC  under18  1992.0    118636.0   
..                    ...          ...      ...     ...         ...   
58                 Alaska           AK    total  1995.0    604412.0   
53                 Alaska           AK    total  1994.0    603308.0   
56                 Alaska           AK    total  1993.0    599434.0   
50                 Alaska           AK    total  1992.0    588736.0   
55                 Alaska           AK    total  1991.0    570193.0   
48                 Alaska           AK    total  1990.0    553290.0   
63                 Alaska           AK  under18  1998.0    192636.0   
66                 Alaska           AK  under18  1999.0    191422.0   
69                 Alaska           AK  under18  2000.0    190615.0   
71                 Alaska           AK  under18  2001.0    188771.0   
73                 Alaska           AK  under18  2002.0    188482.0   
92                 Alaska           AK  under18  2011.0    188329.0   
62                 Alaska           AK  under18  1997.0    188280.0   
94                 Alaska           AK  under18  2012.0    188162.0   
86                 Alaska           AK  under18  2013.0    188132.0   
90                 Alaska           AK  under18  2010.0    187902.0   
54                 Alaska           AK  under18  1994.0    187439.0   
57                 Alaska           AK  under18  1993.0    187190.0   
75                 Alaska           AK  under18  2003.0    186843.0   
89                 Alaska           AK  under18  2009.0    186351.0   
77                 Alaska           AK  under18  2004.0    186335.0   
81                 Alaska           AK  under18  2006.0    185580.0   
61                 Alaska           AK  under18  1996.0    185360.0   
79                 Alaska           AK  under18  2005.0    185304.0   
59                 Alaska           AK  unde